In [1]:
import pandas as pd

import tyr

# Required to get relative path of test datasets
import os

from pprint import pprint

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
# Load schema from pkl
source = tyr.lineage.schema.core.load_schema_from_pkl(
    os.path.abspath(os.path.join(os.getcwd(), "..", "tests/saved_schema/source.pkl"))
)
staging = tyr.lineage.schema.core.load_schema_from_pkl(
    os.path.abspath(os.path.join(os.getcwd(), "..", "tests/saved_schema/staging.pkl"))
)

In [3]:
# Defining connection
conn = tyr.database.connections.Connection(
    name="test",
    syntax="duckdb",
    database=os.path.abspath(os.path.join(os.getcwd(), "..", "tests/test.duckdb")),
    read_only=False,
)

In [4]:
# Confirm that the tables from 1.2 still exist in the database
display(conn.tables())

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,test,singapore,car_location,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,test,singapore,car_telemetry,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,test,singapore,circuits,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,test,singapore,meetings,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,test,singapore,race_control,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,test,singapore,results,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,test,singapore,sessions,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,test,singapore,session_status,BASE TABLE,None,None,None,None,None,YES,NO,None,None
8,test,singapore,track_status,BASE TABLE,None,None,None,None,None,YES,NO,None,None
9,test,singapore,weather,BASE TABLE,None,None,None,None,None,YES,NO,None,None


# Understanding Context

The interpreter treats column objects differently, depending on the context in which they are called.
Within their parent table, the full query required to create the column will be returned. Within
the context of the `lineage.columns.Select` object, only the column name prefixed with the 
parent table name will be returned.

In [5]:
# Context example

print(rf"Within parent table context: {staging.tables.car_telemetry.columns.kmh.sql}")

print("\n")

print(
    rf"Within Select context: {tyr.lineage.columns.Select(staging.tables.car_telemetry.columns.kmh).sql}"
)

Within parent table context: ROUND(TRY_CAST("Speed" AS DECIMAL(15, 5)), 1)


Within Select context: car_telemetry.kmh


Note here that "Speed" is the name of the column in the source file being aliased to kmh.
If you called this column without using a Select object in a new table, the query would fail.

Very important note: Any column assigned as part of the `columns`, `primary_key`, or `event_time` attribute will have their `current_table` attribute overwritten.
Ensure these columns are not within their original context to ensure the original column's `current_table` attribute remains correct

________________________________________________________________________________________
# Creating a new table

In [6]:
# Lets do some examples with the car_telemetry table with a new table called car_telemetry_analysis
# As an example, we will retrieve the average speed in kmh, for each gear, for each available driver

# Best practice is to define the source as a separate object, as most attributes will rely on it
# and it allows ease of re-use of the Select object
car_telemetry_analysis_source = tyr.lineage.tables.Select(staging.tables.car_telemetry)

# Create a new core table
car_telemetry_analysis = tyr.lineage.tables.Core(
    # Give it a name. Best practice is to use the same name as the variable
    name="car_telemetry_analysis",
    # The source will be our previously defined source object
    source=car_telemetry_analysis_source,
    # We want to get the average speed over the static primary key of the car_telemetry table
    # We will first retrieve the static primary key using a macro. The select_static_primary_key
    # retrieves all the primary key columns excluding the event_time from the source table
    # and returns them as lineage.columns.Core objects
    columns=tyr.lineage.macros.columns.select_static_primary_key(
        car_telemetry_analysis_source
    )
    + tyr.lineage.core.ColumnList(
        [
            # We also want to perform this over the n_gear column as well. It is not a part of the static primary key
            # so we will have to bring it in separately
            tyr.lineage.columns.Core(
                name=car_telemetry_analysis_source.columns.n_gear.name,
                source=tyr.lineage.columns.Select(
                    car_telemetry_analysis_source.columns.n_gear
                ),
            ),
            # Finally, we require the average_kmh. We can achieve this using the aggregate.Average function
            tyr.lineage.columns.Core(
                name="average_kmh",
                source=tyr.lineage.functions.aggregate.Average(
                    car_telemetry_analysis_source.columns.kmh
                ),
            ),
        ]
    ),
    # We need to assign the primary key to the table. This is achieved by copying the static primary key columns
    # in the column attribute and the n_gear column as we want to partition over n_gear as well.
    primary_key=tyr.lineage.macros.columns.select_static_primary_key(
        car_telemetry_analysis_source
    )
    + tyr.lineage.core.ColumnList(
        [
            tyr.lineage.columns.Core(
                name=car_telemetry_analysis_source.columns.n_gear.name,
                source=tyr.lineage.columns.Select(
                    car_telemetry_analysis_source.columns.n_gear
                ),
            ),
        ]
    ),
    # Finally, to let the table know to group by the primary key, we need to set group_by=True
    group_by=True,
)

# Execute the query and check the output to ensure it seems correct
display(conn.execute(car_telemetry_analysis.sql).df())

,driver_number,n_gear,average_kmh
0,4,0,0.496366
1,4,1,52.428676
2,4,2,91.924752
3,4,3,128.131872
4,4,4,175.163710
5,4,5,208.123782
6,4,6,238.447406
7,4,7,272.832154
8,4,8,297.107759
9,44,0,0.335976


In [7]:
# This is great, but we don't know who these drivers are. There is more information in the results table
display(conn.execute(staging.tables.results.sql).df())

,session_key,driver_number,broadcast_name,abbreviation,driver_id,team_name,team_colour,team_id,first_name,last_name,full_name,headshot_url,country_code,position,classified_position,grid_position,qualifying_1,qualifying_2,qualifying_3,classified_time,session_status,points,laps_completed
0,9165,24,G ZHOU,ZHO,zhou,Alfa Romeo,C92D4B,alfa,Guanyu,Zhou,Guanyu Zhou,https://www.formula1.com/content/dam/fom-websi...,CHN,12.0,12.0,19.0,NaT,NaT,NaT,0 days 00:01:23.649000,Finished,0,62
1,9165,55,C SAINZ,SAI,sainz,Ferrari,F91536,ferrari,Carlos,Sainz,Carlos Sainz,https://www.formula1.com/content/dam/fom-websi...,ESP,1.0,1.0,1.0,NaT,NaT,NaT,0 days 01:46:37.418000,Finished,25,62
2,9165,44,L HAMILTON,HAM,hamilton,Mercedes,6CD3BF,mercedes,Lewis,Hamilton,Lewis Hamilton,https://www.formula1.com/content/dam/fom-websi...,GBR,3.0,3.0,5.0,NaT,NaT,NaT,0 days 00:00:01.269000,Finished,16,62
3,9165,77,V BOTTAS,BOT,bottas,Alfa Romeo,C92D4B,alfa,Valtteri,Bottas,Valtteri Bottas,https://www.formula1.com/content/dam/fom-websi...,FIN,17.0,NaN,16.0,NaT,NaT,NaT,NaT,Retired,0,51
4,9165,31,E OCON,OCO,ocon,Alpine,2293D1,alpine,Esteban,Ocon,Esteban Ocon,https://www.formula1.com/content/dam/fom-websi...,FRA,18.0,NaN,8.0,NaT,NaT,NaT,NaT,Retired,0,42
5,9165,14,F ALONSO,ALO,alonso,Aston Martin,358C75,aston_martin,Fernando,Alonso,Fernando Alonso,https://www.formula1.com/content/dam/fom-websi...,ESP,15.0,15.0,7.0,NaT,NaT,NaT,0 days 00:01:27.603000,Finished,0,62
6,9165,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,https://www.formula1.com/content/dam/fom-websi...,NED,5.0,5.0,11.0,NaT,NaT,NaT,0 days 00:00:21.441000,Finished,10,62
7,9165,2,L SARGEANT,SAR,sargeant,Williams,37BEDD,williams,Logan,Sargeant,Logan Sargeant,https://www.formula1.com/content/dam/fom-websi...,USA,14.0,14.0,18.0,NaT,NaT,NaT,0 days 00:01:26.889000,Finished,0,62
8,9165,63,G RUSSELL,RUS,russell,Mercedes,6CD3BF,mercedes,George,Russell,George Russell,https://www.formula1.com/content/dam/fom-websi...,GBR,16.0,16.0,2.0,NaT,NaT,NaT,NaT,Finished,0,61
9,9165,20,K MAGNUSSEN,MAG,kevin_magnussen,Haas F1 Team,B6BABD,haas,Kevin,Magnussen,Kevin Magnussen,https://www.formula1.com/content/dam/fom-websi...,DEN,10.0,10.0,6.0,NaT,NaT,NaT,0 days 00:01:12.116000,Finished,1,62


In [8]:
# Let's use a join object to combine our metrics and the results table
# We will also require use of the ctes parameter of the table Core object

left = tyr.lineage.tables.Select(car_telemetry_analysis)
right = tyr.lineage.tables.Select(staging.tables.results)

analysis_results_join = tyr.lineage.joins.Join(
    join_expression=tyr.lineage.expressions.LeftJoin(left=left, right=right),
    condition=tyr.lineage.core.Condition(
        checks=[
            tyr.lineage.expressions.Equal(
                tyr.lineage.columns.Select(left.columns.driver_number),
                tyr.lineage.columns.Select(right.columns.driver_number),
            )
        ]
    ),
)

joined_analysis_results_1 = tyr.lineage.tables.Core(
    name="joined_analysis_results",
    ctes=tyr.lineage.core.TableList([car_telemetry_analysis]),
    source=analysis_results_join,
    columns=tyr.lineage.core.ColumnList(
        [
            tyr.lineage.columns.Core(
                name=name,
                source=analysis_results_join.join_expression.right.columns[name],
            )
            for name in ["team_name", "broadcast_name"]
        ]
    )
    + tyr.lineage.macros.columns.select_all(analysis_results_join.join_expression.left),
)

display(conn.execute(joined_analysis_results_1.sql + " ORDER BY 3,4 ASC").df())

,team_name,broadcast_name,driver_number,n_gear,average_kmh
0,McLaren,L NORRIS,4,0,0.496366
1,McLaren,L NORRIS,4,1,52.428676
2,McLaren,L NORRIS,4,2,91.924752
3,McLaren,L NORRIS,4,3,128.131872
4,McLaren,L NORRIS,4,4,175.163710
5,McLaren,L NORRIS,4,5,208.123782
6,McLaren,L NORRIS,4,6,238.447406
7,McLaren,L NORRIS,4,7,272.832154
8,McLaren,L NORRIS,4,8,297.107759
9,Mercedes,L HAMILTON,44,0,0.335976


In [9]:
# There is no single method of achieving this result. Here is another method using a Subquery

left = tyr.lineage.tables.Subquery(car_telemetry_analysis)
right = tyr.lineage.tables.Select(staging.tables.results)

analysis_results_join = tyr.lineage.joins.Join(
    join_expression=tyr.lineage.expressions.LeftJoin(left=left, right=right),
    condition=tyr.lineage.core.Condition(
        checks=[
            tyr.lineage.expressions.Equal(
                tyr.lineage.columns.Select(left.columns.driver_number),
                tyr.lineage.columns.Select(right.columns.driver_number),
            )
        ]
    ),
)

joined_analysis_results_source = tyr.lineage.tables.Core(
    name="joined_analysis_results_source",
    source=analysis_results_join,
    columns=tyr.lineage.macros.columns.select_all(analysis_results_join),
    primary_key=tyr.lineage.macros.columns.select_all(
        analysis_results_join,
        filter_regex=rf"^(?!{'|'.join([column for column in analysis_results_join.join_expression.left.columns.list_names_()])}).*",
    ),
)

joined_analysis_results_source_subquery = tyr.lineage.tables.Subquery(
    source=joined_analysis_results_source,
)

joined_analysis_results_2 = tyr.lineage.tables.Core(
    name="joined_analysis_results",
    source=joined_analysis_results_source_subquery,
    columns=tyr.lineage.core.ColumnList(
        tyr.lineage.macros.columns.select_all(joined_analysis_results_source_subquery)[
            ["team_name", "broadcast_name", "driver_number", "n_gear", "average_kmh"]
        ]
    ),
    inherit_primary_key=True,
)

print(joined_analysis_results_2.sql)
display(conn.execute(joined_analysis_results_2.sql + " ORDER BY 3, 4 ASC").df())

SELECT joined_analysis_results_source.team_name AS team_name,
       joined_analysis_results_source.broadcast_name AS broadcast_name,
       joined_analysis_results_source.driver_number AS driver_number,
       joined_analysis_results_source.n_gear AS n_gear,
       joined_analysis_results_source.average_kmh AS average_kmh
FROM
  (SELECT car_telemetry_analysis.driver_number AS driver_number,
          car_telemetry_analysis.n_gear AS n_gear,
          car_telemetry_analysis.average_kmh AS average_kmh,
          results.session_key AS session_key,
          results.broadcast_name AS broadcast_name,
          results.abbreviation AS abbreviation,
          results.driver_id AS driver_id,
          results.team_name AS team_name,
          results.team_colour AS team_colour,
          results.team_id AS team_id,
          results.first_name AS first_name,
          results.last_name AS last_name,
          results.full_name AS full_name,
          results.headshot_url AS headshot_url,
   

,team_name,broadcast_name,driver_number,n_gear,average_kmh
0,McLaren,L NORRIS,4,0,0.496366
1,McLaren,L NORRIS,4,1,52.428676
2,McLaren,L NORRIS,4,2,91.924752
3,McLaren,L NORRIS,4,3,128.131872
4,McLaren,L NORRIS,4,4,175.163710
5,McLaren,L NORRIS,4,5,208.123782
6,McLaren,L NORRIS,4,6,238.447406
7,McLaren,L NORRIS,4,7,272.832154
8,McLaren,L NORRIS,4,8,297.107759
9,Mercedes,L HAMILTON,44,0,0.335976


In [10]:
# We can confirm that these two methods produce the same result

conn.execute(joined_analysis_results_1.sql).df().equals(
    conn.execute(joined_analysis_results_2.sql).df()
)

True

In [11]:
conn.close()